In [1]:
from plots import plot_curvatures

In [4]:
# Good to show the impact of both!!!
methods = ['StdGP', 'Double Tournament (Prob=1)']
plot_curvatures('USCrime', methods, run_nr=20, base_path='/home/ines/Documents/tese/tiny_gp', saving_path='/home/ines/Documents/tese/dissertation/novathesis/Chapters/Figures/curvature-DT-StdGP-USCrime.png')

METHOD: StdGP
(32, 13)
10     (((((x2) / (x1)) + ((x9) / (x8))) * (((((((((x...
150    (((((x2) / (x7)) + ((x9) / (x8))) * (((x5) * (...
500    (((((x2) / (x7)) + ((x9) / (x8))) * (((x5) * (...
Name: 0, dtype: object


/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



METHOD: Double Tournament (Prob=1)
(32, 13)
10     ((((((x10) / ((x2) / (x2))) + (x9)) + ((x5) + ...
150    ((((x10) / (x1)) / (x3)) + ((((((((x11) + ((x1...
500    ((((x10) + ((x10) + ((((x10) + (x10)) + (x10))...
Name: 0, dtype: object


/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [2]:
# Good to show impact of curvature!!!
methods = ['StdGP', 'Double Tournament (Prob=1)']
plot_curvatures('Istanbul', methods, run_nr=29, base_path='/home/ines/Documents/tese/tiny_gp', saving_path='/home/ines/Documents/tese/dissertation/novathesis/Chapters/Figures/curvature-DT-StdGP.png')

METHOD: StdGP
(375, 7)
10     ((((x7) - (x6)) * ((x7) - (x3))) * ((x5) - (((...
150    ((x5) * ((x5) - (x7))) * ((((((x6) - (x5)) + (...
500    ((x5) * ((x5) - (x7))) * ((((((x6) - (x5)) + (...
Name: 0, dtype: object


/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



METHOD: Double Tournament (Prob=1)
(375, 7)
10                         ((x5) * (x2)) - ((x5) * (x2))
150    ((((x6) - (x2)) * (((((x6) - (x6)) - (x4)) * (...
500    (((x6) * ((((((x2) + (x1)) * ((x7) - (x2))) / ...
Name: 0, dtype: object


/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [4]:
# Good to show impact of features!!!
methods = ['StdGP', 'Double Tournament (Prob=1)']
plot_curvatures('Istanbul', methods, run_nr=4, base_path='/home/ines/Documents/tese/tiny_gp', saving_path='/home/ines/Documents/tese/dissertation/novathesis/Chapters/Figures/curvature-DT-StdGP-Istanbul-feats.png')

METHOD: StdGP
(375, 7)
10     (((((x2) / (x4)) - ((x6) + (x5))) * (((x6) * (...
250    (((((x2) / (x4)) - ((x6) + (x2))) * (((x6) * (...
500    (((((x2) / (x4)) - ((x6) + (x2))) * (((x6) * (...
Name: 0, dtype: object


/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



METHOD: Double Tournament (Prob=1)
(375, 7)
10     (((x3) - (x6)) * ((x6) * (x7))) * ((x6) * ((x7...
250    ((x6) * (((x4) * (x6)) * (x6))) * (((((x7) - (...
500    ((x6) * (((x4) * (x6)) * (x6))) * (((((x7) - (...
Name: 0, dtype: object


/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [3]:
methods = ['Double Tournament (Prob=1)', 'Subsampled', 'Oversampled']
plot_curvatures('USCrime', methods, run_nr=20, base_path='/home/ines/Documents/tese/tiny_gp')

METHOD: Double Tournament (Prob=1)
(32, 13)
10     ((((((x10) / ((x2) / (x2))) + (x9)) + ((x5) + ...
250    ((((x10) / (x2)) / (x3)) + ((((((((x11) + ((((...
500    ((((x10) + ((x10) + ((((x10) + (x10)) + (x10))...
Name: 0, dtype: object


/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



METHOD: Subsampled
(16, 13)
10     (x3) + ((x13) + ((((x2) + ((x12) + ((x7) + (((...
250    ((((((((x7) + (((x12) / (x12)) + (x13))) + (((...
500    (((((((((x7) + ((((x7) / ((x4) + (x4))) + ((x2...
Name: 0, dtype: object
METHOD: Oversampled
(64, 13)
10                       (((x12) / (x6)) / (x11)) / (x1)
250    (((((((((x1) + ((x1) / (((x1) + (x1)) / (x3)))...
500    ((((((((x7) / (x1)) + ((((x1) + (x4)) + (x6)) ...
Name: 0, dtype: object


/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/home/ines/Documents/tese/tiny_gp/print_functions/plots.py:65: FutureWarning:

Series.__getitem__ treating ke